In [1]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise import Trainset

from collections import defaultdict
from operator import itemgetter
import heapq

import os
import csv

import pandas as pd

# Import CSV 

In [2]:
# UsersDF = pd.read_csv('../raw_data/users_cleaned.csv')
# AnimesDF = pd.read_csv('../raw_data/anime_cleaned.csv')
ScoresDF = pd.read_csv('../raw_data/animelists_cleaned.csv')

In [3]:
ScoresDF.shape

(31284030, 11)

In [4]:
AnimesDF = pd.read_csv('../raw_data/anime_cleaned.csv')

In [24]:
AnimesDF[AnimesDF["title"]=="Clannad: After Story"]

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
1366,4181,Clannad: After Story,Clannad ~After Story~,CLANNAD〜AFTER STORY〜 クラナド アフターストーリー,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Visual novel,24,Finished Airing,...,Fridays at 01:59 (JST),"{'Alternative version': [{'mal_id': 1723, 'typ...","Pony Canyon, TBS, Rakuonsha, Animation Do",Sentai Filmworks,Kyoto Animation,"Slice of Life, Comedy, Supernatural, Drama, Ro...","['""Toki wo Kizamu Uta (時を刻む唄)"" by Lia']","['#1: ""TORCH"" by Lia (eps 1-21)', '#2: ""Chiisa...",24.0,2008.0


In [5]:
score_anime=ScoresDF.merge(AnimesDF,on="anime_id")

In [ ]:
score_anime.describe()

In [27]:
df3 = score_anime[score_anime["my_score"]>0][["username","anime_id","my_score","title"]]

In [30]:
df3[df3["username"]=="Evilrelena"]

,username,anime_id,my_score,title
526,Evilrelena,21,10,One Piece
45367,Evilrelena,59,7,Chobits
82535,Evilrelena,74,9,Gakuen Alice
91080,Evilrelena,120,9,Fruits Basket
125990,Evilrelena,210,8,Ranma ½
...,...,...,...,...
30673776,Evilrelena,537,7,Tekken
30928252,Evilrelena,234,8,Dan Doh!!
30951561,Evilrelena,1804,6,Koko wa Green Wood
31033526,Evilrelena,378,7,eX-Driver the Movie


In [6]:
score_anime_selected = score_anime[score_anime["popularity"]<70][score_anime["my_score"]>0][["username","anime_id","my_score","title"]]

/var/folders/yr/g3l3s88n1fj3tj6803gfpmm80000gn/T/ipykernel_50814/398666508.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  score_anime_selected = score_anime[score_anime["popularity"]<70][score_anime["my_score"]>0][["username","anime_id","my_score","title"]]


In [7]:
score_anime_selected.shape

(2344515, 4)

In [12]:
score_anime_selected.to_csv("score_anime_selected.csv",index=False)

In [13]:
score_anime_selected

,username,anime_id,my_score,title
0,karthiga,21,9,One Piece
1,Damonashu,21,10,One Piece
2,bskai,21,8,One Piece
3,Slimak,21,10,One Piece
5,kioniel,21,9,One Piece
...,...,...,...,...
19296613,Whirson,28171,8,Shokugeki no Souma
19296614,Morse56,28171,10,Shokugeki no Souma
19296615,arya333,28171,8,Shokugeki no Souma
19296616,jurandyr,28171,8,Shokugeki no Souma


# Build trainset

In [ ]:
ScoresDF["my_score"].value_counts()

In [ ]:
ScoresDF.shape

In [ ]:
ScoresDF.head()

In [ ]:
ScoresDF_selected= ScoresDF[ScoresDF["my_score"]>0][["username","anime_id","my_score","my_last_updated"]]

In [ ]:
ScoresDF_selected.shape

In [ ]:
#small_fraction = 0.0001
#ScoresDF_selected_small = ScoresDF_selected.sample(int(len(ScoresDF_selected)*small_fraction))
#ScoresDF_selected_small.shape

# Transform dataframe to surprise trainset

In [9]:
reader = Reader(rating_scale=(0, 10))
scoredata = Dataset.load_from_df(score_anime_selected[['username', 'anime_id', 'my_score']], reader)
trainset = scoredata.build_full_trainset()

In [ ]:
print(trainset)

# Try KNNBasic

In [10]:
knn_predictor = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': False
        })
        
knn_predictor.fit(trainset)
similarity_matrix_small = knn_predictor.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
similarity_matrix


In [11]:
# save numpy array as npy file
from numpy import asarray
from numpy import save

# save to npy file
save('similarity_matrix_small.npy', similarity_matrix_small)

# Create Movie Dictionary 

In [ ]:
AnimesDF.head()

In [ ]:
animeID_to_name = score_anime_selected.set_index('anime_id')['title'].to_dict()
animeID_to_name

# Set variables

In [ ]:
ScoresDF_selected.groupby("username").agg({"anime_id":"count"}).describe()

In [ ]:
#ScoresDF_selected.tail(15)
ScoresDF[["username","anime_id","my_watched_episodes", "my_score"]].tail(10)

In [ ]:
# Pick a random user name.
# Play around and see how the final recommendations change
# depending on the user! 1-610
test_subject = 'karthiga'

# Get the top K items user rated
k = 20

In [ ]:
# When using Surprise, there are RAW and INNER IDs.
# Raw IDs are the IDs, strings or numbers, you use when
# creating the trainset. The raw ID will be converted to
# an unique integer Surprise can more easily manipulate
# for computations.
#
# So in order to find an user inside the trainset, you
# need to convert their RAW ID to the INNER Id. Read
# here for more info https://surprise.readthedocs.io/en/stable/FAQ.html#what-are-raw-and-inner-ids
test_subject_iid = trainset.to_inner_uid(test_subject)
test_subject_iid



In [ ]:
test_subject_ratings = trainset.ur[test_subject_iid]
test_subject_ratings

In [ ]:
# Get the top K items we rated
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])
k_neighbors

In [ ]:
test_subject_iid

In [ ]:
itemID_tmp =6
rating_temp=10.0

In [ ]:
similarity_matrix[itemID_tmp]

In [ ]:
# Default dict is basically a standard dictionary,
# the difference beeing that it doesn't throw an error
# when trying to access a key which does not exist,
# instead a new entry, with that key, is created.
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue
  


In [ ]:
candidates

In [ ]:
similarity_matrix[1]

In [ ]:
# Utility we'll use later.
def getAnimeName(animeID):
  if int(animeID) in animeID_to_name:
    return animeID_to_name[int(animeID)]
  else:
      return ""

In [ ]:
getAnimeName("5231")

In [ ]:
# Build a dictionary of movies the user has watched
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1

# Add items to list of user's recommendations
# If they are similar to their favorite movies,
# AND have not already been watched.
recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched:
    recommendations.append(getAnimeName(trainset.to_raw_iid(itemID)))
    position += 1
    if (position > 10): break # We only want top 10

for rec in recommendations:
  print("Anime: ", rec)

In [ ]:
watched